<a href="https://colab.research.google.com/github/Aaron-David-HdM/Applied-Data-Analytics/blob/main/UseCase_1/Datenimport_und_Bereinigung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Zeigt Inhalte des gewählten Ordners**

In [ ]:
ls

Applied-Data-Analytics/  sample_data/


**Verknüpfung zum GitHub-Repository**

In [282]:
#!git clone https://github.com/Aaron-David-HdM/Applied-Data-Analytics.git

fatal: not a git repository (or any of the parent directories): .git


**Wechselt zum lokalen Repository-Verzeichnis und holt Daten vom GitHub-Repo**

In [ ]:
%cd /content/Applied-Data-Analytics/
!git pull

**Bibliotheken-Import, Wechsel zum lokalen Repository und lädt die CSV-Dateien.**

In [324]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

%cd /content/Applied-Data-Analytics/UseCase_1

filepath_1 = 'Zielgruppenuebersicht_1_August_2021.csv' # Gesamtzahl der Nutzer,18-24 sowie 25-34 jährige Nutzer
filepath_2 = 'Zielgruppenuebersicht_2_August_2021.csv' # 35-44,	45-54 und 55-64 jährige Nutzer sowie Nutzer über 65 Jahren


df_1 = pd.read_csv(filepath_1, decimal=',')
df_2 = pd.read_csv(filepath_2, decimal=',')

/content/Applied-Data-Analytics/UseCase_1


**Daten Bereinigung**

Google Analytics erlaubt nur die Verknüpfung von bis zu 4 verschiedene Segmenten bei der Erzeugung der Daten. Somit waren wir gezwungen, zwei Tabellen anzulegen und diese im Anschluss miteinander zu 'mergen'.


**Löschen von NaN-Werten und der 'Zeitraum'-Column sowie dem Ersetzen der Punkte bei den Zahlenwerten**

In [ ]:
new_df_1 = df_1.dropna()
new_df_1 = df_1.drop(columns=['Zeitraum'])
new_df_1['Nutzer'] = new_df_1['Nutzer'].str.replace('.', '')

new_df_2 = df_2.dropna()
new_df_2 = df_2.drop(columns=['Zeitraum'])
new_df_2['Nutzer'] = new_df_2['Nutzer'].str.replace('.', '')

**Columns umbenannt und neu gruppiert.**

In [312]:
new_df_1 = new_df_1.rename(columns={'Segment':'Altersgruppe', 'Index: Tag':'Tag'})
new_df_1 = new_df_1.groupby(['Tag', 'Altersgruppe'])['Nutzer'].first().unstack()
new_df_1.columns = ['Alle Nutzer', '18-24', '25-34']

new_df_2 = new_df_2.rename(columns={'Segment':'Altersgruppe', 'Index: Tag':'Tag'})
new_df_2 = new_df_2.groupby(['Tag', 'Altersgruppe'])['Nutzer'].first().unstack()
new_df_2.columns = ['35-44', '45-54', '55-64', '65+']

**Nutzerzahlen zu Integern konvertiert.**

In [315]:
new_df_1[['18-24', '25-34', 'Alle Nutzer']] = new_df_1[['18-24', '25-34', 'Alle Nutzer']].apply(np.int64)

new_df_2[['35-44', '45-54', '55-64', '65+']] = new_df_2[['35-44', '45-54', '55-64', '65+']].apply(np.int64)

**Die zwei Dataframes zu einem Dataframe zusammengeführt.**

In [319]:
df = new_df_1.merge(new_df_2, how='inner', left_index=True, right_index=True)
df

,Alle Nutzer,18-24,25-34,35-44,45-54,55-64,65+
Tag,,,,,,,
01.08.21,1420,144,177,96,50,45,23
02.08.21,2505,253,454,211,142,74,50
03.08.21,2545,279,397,201,127,69,61
04.08.21,2518,306,407,225,120,77,37
05.08.21,2502,243,392,232,140,66,52
06.08.21,2066,214,338,162,77,54,37
07.08.21,1284,130,139,84,38,44,22
08.08.21,1409,146,196,90,51,22,23
09.08.21,2309,256,387,186,131,46,58


In [326]:
df.to_csv('Zielgruppenuebersicht_Gruppe_A.csv', sep=';')